# 실습 4: Agent 생성

## 목표
- Agent Factory 생성
- Memory Hooks 구현
- Memory Manager 구현

## 4-1. 환경 변수 로드

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

REGION = os.getenv('AWS_REGION', 'us-west-2')
MEMORY_ID = os.getenv('MEMORY_ID', '')

print(f"🌍 Region: {REGION}")
print(f"🧠 Memory ID: {MEMORY_ID if MEMORY_ID else '미설정'}")

## 4-2. 디렉토리 구조 생성

In [ ]:
import os

# 디렉토리 생성
dirs = [
    'agent/deploy/runtime/agent',
    'agent/deploy/runtime/memory',
]

for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)
    # __init__.py 생성
    init_file = os.path.join(dir_path, '__init__.py')
    if not os.path.exists(init_file):
        open(init_file, 'w').close()

print("✅ 디렉토리 구조 생성 완료")

## 4-3. Memory Manager 생성

In [ ]:
manager_content = f'''"""Memory session management"""

from bedrock_agentcore.memory.session import MemorySession, MemorySessionManager

MEMORY_ID = "{MEMORY_ID}"
REGION = "{REGION}"

_session_manager = None

def get_memory_session_manager() -> MemorySessionManager:
    """Get or create memory session manager"""
    global _session_manager
    if _session_manager is None:
        _session_manager = MemorySessionManager(memory_id=MEMORY_ID, region_name=REGION)
        print(f"✅ Session manager initialized for memory: {{MEMORY_ID}}")
    return _session_manager

def create_memory_session(actorId: str, sessionId: str) -> MemorySession:
    """Create a memory session for the given actor and session"""
    manager = get_memory_session_manager()
    session = manager.create_memory_session(
        actor_id=actorId,
        session_id=sessionId
    )
    print(f"🔧 Memory session created (actor: {{actorId}}, session: {{sessionId}})")
    return session
'''

with open('agent/deploy/runtime/memory/manager.py', 'w', encoding='utf-8') as f:
    f.write(manager_content)

print("✅ memory/manager.py 생성 완료")

## 4-4. Memory Hooks 생성

In [ ]:
hooks_content = '''"""Memory hooks for agent"""

from strands import HookProvider
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.session import MemorySession

class ShortTermMemoryHookProvider(HookProvider):
    """Short-term memory hook"""
    def __init__(self, session: MemorySession):
        self.session = session
    
    def on_agent_start(self, agent, input_data):
        """Load short-term memory"""
        pass
    
    def on_agent_end(self, agent, output_data):
        """Save short-term memory"""
        pass

class LongTermMemoryHookProvider(HookProvider):
    """Long-term memory hook"""
    def __init__(self, memory_id: str, memory_client: MemoryClient):
        self.memory_id = memory_id
        self.memory_client = memory_client
    
    def on_agent_start(self, agent, input_data):
        """Load long-term memory"""
        pass
    
    def on_agent_end(self, agent, output_data):
        """Save long-term memory"""
        pass
'''

with open('agent/deploy/runtime/memory/hooks.py', 'w', encoding='utf-8') as f:
    f.write(hooks_content)

print("✅ memory/hooks.py 생성 완료")

## 4-5. Agent Factory 생성

In [ ]:
factory_content = f'''"""Agent factory for creating configured agents"""

from strands import Agent
from strands.models import BedrockModel
from bedrock_agentcore.memory import MemoryClient

MODEL_ID = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"
REGION = "{REGION}"
MEMORY_ID = "{MEMORY_ID}"
AGENT_NAME = "FireCommandAssistant"

AGENT_SYSTEM_PROMPT = """당신은 소방서 화재 대응 지휘를 지원하는 전문 AI 어시스턴트입니다.

**핵심 임무:**
1. 화재 현장의 전술적 정보 제공
2. 신속한 의사결정 지원
3. 대응 자원 배치 최적화 제안

**절대 규칙:**
1. 모든 응답은 한글로 작성
2. 도구 사용 시 목적과 결과 명시
3. 소방서 검색 시 <event type="geocode">위도,경도,소방서이름</event> 형식 반환
4. 기상 정보 조회 시 <event type="windy">위도,경도,온도,풍속,풍향,습도,기압,주소</event> 형식 반환
"""

def create_agent(actorId: str, sessionId: str) -> Agent:
    """메모리와 도구가 설정된 에이전트를 생성합니다"""
    import sys
    import os
    sys.path.insert(0, os.path.dirname(__file__))
    
    from memory.manager import create_memory_session
    from memory.hooks import ShortTermMemoryHookProvider, LongTermMemoryHookProvider
    
    userSession = create_memory_session(actorId, sessionId)
    memory_client = MemoryClient(region_name=REGION)
    model = BedrockModel(model_id=MODEL_ID)
    
    agent = Agent(
        name=AGENT_NAME,
        model=model,
        system_prompt=AGENT_SYSTEM_PROMPT,
        hooks=[
            ShortTermMemoryHookProvider(userSession),
            LongTermMemoryHookProvider(MEMORY_ID, memory_client)
        ],
        tools=[],  # lab5에서 추가
        state={{"actor_id": actorId, "session_id": sessionId}}
    )
    
    print(f"✅ 에이전트 생성 완료 (actor: {{actorId}}, session: {{sessionId}})")
    return agent
'''

with open('agent/deploy/runtime/agent/factory.py', 'w', encoding='utf-8') as f:
    f.write(factory_content)

print("✅ agent/factory.py 생성 완료")

## ✅ 실습 4 완료!

### 생성된 파일
- `agent/deploy/runtime/memory/manager.py`
- `agent/deploy/runtime/memory/hooks.py`
- `agent/deploy/runtime/agent/factory.py`

### 다음 단계
👉 **lab5_create_tools.ipynb**